# APAs Network Graph Creation

This code contains all that it needed to recreate the network graph of apas and meps in both 9th and 10th terms. 

# 1. Set-Up

## 1.1 Import Libraries

In [1]:
import pandas as pd
import json
import os